In [6]:
import cv2
import numpy as np
import glob

# ====== 参数设置 ======
image_dir = "E:/01_Graduate_projects/Cellular_structures/Multi-functional_design/exp/camera_calib/imgs/*.jpg"  # 你的图片路径
pattern_size = (11, 8)
square_size = 20

# ====== 世界坐标系坐标准备 ======
objp = np.zeros((pattern_size[0]*pattern_size[1], 3), np.float32)
objp[:,:2] = np.mgrid[0:pattern_size[0], 0:pattern_size[1]].T.reshape(-1,2) * square_size

# ====== 角点数据收集 ======
obj_points = []
img_points = []

images = glob.glob(image_dir)
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    ret, corners = cv2.findChessboardCorners(gray, pattern_size, None)
    
    if ret:
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
        corners_refined = cv2.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria)
        
        obj_points.append(objp)
        img_points.append(corners_refined)
        
        # ======== 关键修改1：显示时缩小窗口 ========
        h, w = img.shape[:2]
        cv2.namedWindow('Detected Corners', cv2.WINDOW_NORMAL)
        cv2.resizeWindow('Detected Corners', int(w*0.25), int(h*0.25))  # 窗口缩小至25%
        cv2.drawChessboardCorners(img, pattern_size, corners_refined, ret)
        cv2.imshow('Detected Corners', img)
        cv2.waitKey(500)
    else:
        print(f"未检测到角点：{fname}")

cv2.destroyAllWindows()

# ====== 相机标定 ======
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(obj_points, img_points, gray.shape[::-1], None, None)

# ====== 打印标定参数 ====== [1,4](@ref)
print("\n====== 相机内参矩阵 ======")
print("K = \n", mtx)  # 内参矩阵 (3x3)
print("\n====== 畸变系数 ======")
print("D = ", dist.ravel())  # 畸变系数 k1,k2,p1,p2,k3,...

# 打印外参（示例：第一张图的旋转矩阵和平移向量）[4,7](@ref)
if len(rvecs) > 0:
    # 将旋转向量转换为旋转矩阵
    R,_ = cv2.Rodrigues(rvecs[0])
    print("\n====== 第一张图的外参 ======")
    print("旋转矩阵 R = \n", R)
    print("平移向量 t = \n", tvecs[0].T)


# ====== 标定结果评估 ======
mean_error = 0
for i in range(len(obj_points)):
    imgpoints2, _ = cv2.projectPoints(obj_points[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv2.norm(img_points[i], imgpoints2, cv2.NORM_L2)/len(imgpoints2)
    mean_error += error
print(f"重投影误差：{mean_error/len(obj_points):.4f} 像素")

# ====== 保存标定参数 ======
np.savez("calibration_params.npz", mtx=mtx, dist=dist)

# ====== 畸变矫正示例 ======
img = cv2.imread(images[0])
h, w = img.shape[:2]
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

# 矫正图像
dst = cv2.undistort(img, mtx, dist, None, newcameramtx)
x,y,w_roi,h_roi = roi
dst = dst[y:y+h_roi, x:x+w_roi]

# ======== 关键修改2：所有显示窗口统一缩放 ========
def show_scaled(winname, image, scale=0.5):
    """显示缩放窗口的通用函数"""
    h, w = image.shape[:2]
    cv2.namedWindow(winname, cv2.WINDOW_NORMAL)
    cv2.resizeWindow(winname, int(w*scale), int(h*scale))
    cv2.imshow(winname, image)

# 显示原始和矫正图像
show_scaled('Original', img)
show_scaled('Undistorted', dst)

cv2.waitKey(0)
cv2.destroyAllWindows()


====== 相机内参矩阵 ======
K = 
 [[1.41525468e+03 0.00000000e+00 2.02481469e+03]
 [0.00000000e+00 1.42088633e+03 1.13036169e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]

====== 畸变系数 ======
D =  [ 0.00251581 -0.13991614 -0.00167196  0.00049613  0.04186309]

====== 第一张图的外参 ======
旋转矩阵 R = 
 [[ 0.99715727  0.05982513 -0.04580758]
 [-0.06162378  0.99734055 -0.03891435]
 [ 0.0433577   0.04162657  0.99819204]]
平移向量 t = 
 [[-91.65338371 -95.39480027 322.26126894]]
重投影误差：0.1953 像素
